In [25]:
import os

# print the current working directory
print('Original working directory:', os.getcwd())

# change the working directory
os.chdir('/home/vsevolod/sat/circuit_improvement')

os.mkdir('/home/vsevolod/sat/circuit_improvement/temp_results')

Original working directory: /home/vsevolod/sat/circuit_improvement


FileExistsError: [Errno 17] File exists: '/home/vsevolod/sat/circuit_improvement/temp_results'

In [92]:
import itertools
from tqdm.notebook import tqdm
from core import CircuitFinder
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
from pandarallel import pandarallel
from scipy.special import comb

pandarallel.initialize(progress_bar=True, nb_workers=4)

dimension = 3
outputs = 3

def find_min_circuit(tables):
    if not hasattr(find_min_circuit, "gates_predict"):
        find_min_circuit.gates_predict = 1

    # print(f"Current gates_pred: {find_min_circuit.gates_predict}\n", end='')
    
    best_circuit = False
    best_gates = 1000
    forbidden_operations = ['0100', '1101', '0010', '1011'] # Bench
    # forbidden_operations = ['0110', '1001'] # AIG


    gates = int(round(find_min_circuit.gates_predict)) + 1

    def get_finder(gates):
        return CircuitFinder(dimension=dimension, number_of_gates=gates, output_truth_tables=tables, forbidden_operations=forbidden_operations)

    def find_curcuit(finder):
        return finder.solve_cnf_formula(solver=None)
    
    while True:
        # print(f"Try {gates} gates")
        circuit_finder = get_finder(gates)
        circuit = find_curcuit(circuit_finder)
        if circuit:
            # print(f"Sucess with {gates}, let's try lower value")
            best_gates = gates
            best_circuit = circuit
            # print(f"Best circuit {best_circuit} saved with {best_gates} gates")
            gates -= 1
        else:
            # print(f"Ok... Did not found circuit with {gates}, the answer is bigger")
            gates += 1
            break
    
    while not best_circuit:
        # print(f"Try {gates} gates")
        circuit_finder = get_finder(gates)
        circuit = find_curcuit(circuit_finder)
        if circuit:
            # print(f"Sucess with {gates}, let's stop the search")
            best_circuit = circuit
            best_gates = gates
            # print(f"Best circuit {best_circuit} saved with {best_gates} gates")
        else:
            # print(f"Not found the answer with {gates} gates, let's find a bigger value")
            gates += 1

    p = 0.0
    find_min_circuit.gates_predict = find_min_circuit.gates_predict * p + best_gates * (1 - p)
    # print(f"Best circuit for {tables} has {best_gates} gates\n", end='') 
    return tables,  best_gates, best_circuit
    

def truth_tables(inputs_count, outputs_count):
    for tables in itertools.combinations(itertools.product('01', repeat=2 ** inputs_count - 1), outputs_count):
        tables = tuple(map(lambda x: '0' + ''.join(x), tables))
        yield tables


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [93]:
tables_series = pd.Series(truth_tables(dimension, outputs))
len(tables_series)

341376

In [94]:
# result = tables_series.parallel_map(find_min_circuit)

In [95]:
import time

start_time = time.perf_counter()

with ProcessPoolExecutor() as pool:
    result = pd.Series(tqdm(pool.map(find_min_circuit, tables_series), total=len(tables_series), desc='Finding circuits'))

end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Process ForkProcess-159:
Process ForkProcess-154:
Process ForkProcess-156:
Process ForkProcess-158:
Process ForkProcess-155:
Process ForkProcess-153:
Process ForkProcess-157:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkProcess-160:
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.11/multiprocessing/process

KeyboardInterrupt: 

In [57]:
# TIME_LIMIT = 1  # seconds

# with ProcessPoolExecutor() as executor:
#     futures = [(table, executor.submit(find_min_circuit, table)) for table in tqdm(tables_series, desc='Submitting tasks')]

#     results = []
#     for table, future in tqdm(futures, total=len(tables_series), desc='Finding circuits'):
#         try:
#             result = future.result(timeout=TIME_LIMIT)
#         except TimeoutError:
#             print(f"Function {tables} time outed")
#             result = None
#         results.append(result)

# # Convert the results to a pandas Series
# result = pd.Series(results)

In [ ]:
i = 200000
result[i]

In [ ]:
with open("AIGdb.txt", "w") as f:
    for tables, gates, circuit in tqdm(result.dropna()):
        f.write(' '.join(tables))
        f.write(f" {gates}\n")
        f.write(str(circuit))
        f.write(f"\n==================\n")

In [ ]:
for tables in itertools.product('01', repeat=4):
    tables, gates, circuit = find_min_circuit([''.join(tables)])
    print(*tables, gates)
    print(circuit)
    print('---------------')
    assert gates <= 2
    